In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
from sklearn.base import TransformerMixin

In [29]:
file_path = r"C:\Users\7J6594897\Desktop\Github Projects\Clustering-CityofAustin311Calls\Data\Austin_311_Public_Data_20231107.csv"
sample_data = pd.read_csv(file_path).sample(100)
sample_data

C:\Users\7J6594897\AppData\Local\Temp\ipykernel_17468\1491414668.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  sample_data = pd.read_csv(file_path).sample(100)


,Service Request (SR) Number,SR Description,Method Received,SR Status,Status Change Date,Created Date,Last Update Date,Close Date,SR Location,Street Number,...,Zip Code,County,State Plane X Coordinate,State Plane Y Coordinate,Latitude Coordinate,Longitude Coordinate,(Latitude.Longitude),Council District,Map Page,Map Tile
477835,22-00064614,Traffic Engineering - General,E-Mail,Open,02/23/2022 10:00:42 AM,02/22/2022 04:39:42 PM,02/23/2022 10:00:42 AM,NaN,"2206 E 51ST ST, AUSTIN, TX 78723",2206.0,...,78723.0,TRAVIS,3.130773e+06,1.008303e+07,30.300753,-97.689756,"(30.30075289, -97.68975582)",1.0,556X,ML25
579557,15-00217204,Bicycle Issues,Phone,Closed,05/10/2016 12:11:28 PM,09/11/2015 10:24:10 AM,05/10/2016 12:11:29 PM,05/10/2016 12:11:28 PM,"2246 GUADALUPE ST, AUSTIN, TX 78705",2246.0,...,78705.0,TRAVIS,3.114379e+06,1.007733e+07,30.286140,-97.742112,"(30.28613957, -97.74211161)",9.0,585E,MJ24
1644292,23-00436886,Traffic Signal - Maintenance,Phone,Closed,09/19/2023 06:33:37 PM,09/19/2023 06:09:16 PM,09/19/2023 06:33:37 PM,09/19/2023 06:33:37 PM,"MC KINNEY FALLS PKWY & E WILLIAM CANNON DR, AU...",NaN,...,78744.0,TRAVIS,3.120563e+06,1.003276e+07,30.163217,-97.725836,"(30.16321718, -97.72583647)",2.0,675U,MK13
424812,21-00533242,Traffic Signal - Maintenance,Mobile Device,Closed,12/06/2021 04:30:41 PM,12/03/2021 04:21:15 PM,12/06/2021 04:30:41 PM,12/06/2021 04:30:41 PM,"HART LN & FAR WEST BLVD, AUSTIN, TX 78731",NaN,...,78731.0,TRAVIS,3.109213e+06,1.010230e+07,30.355108,-97.756644,"(30.35510826, -97.75664414)",10.0,524M,MH29
1119591,18-00235981,Parking Machine Issue,Phone,Closed,07/24/2018 06:06:45 AM,07/21/2018 06:29:50 PM,07/24/2018 06:06:45 AM,07/24/2018 06:06:45 AM,"RIO GRANDE ST & W 6TH ST, AUSTIN, TX 78701",NaN,...,78701.0,TRAVIS,3.112175e+06,1.007139e+07,30.269944,-97.749529,"(30.26994409, -97.74952908)",9.0,585S,MJ22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015199,15-00013966,Flooding Current (Non-Emergency),Spot311 Interface,Closed,02/22/2015 02:02:21 AM,01/22/2015 11:45:52 AM,02/22/2015 01:05:06 AM,02/22/2015 02:02:21 AM,"7900 METROPOLIS DR, AUSTIN, TX",7900.0,...,78744.0,TRAVIS,3.131127e+06,1.004918e+07,30.207670,-97.691182,"(30.20767038, -97.69118227)",2.0,646P,ML17
1494200,23-00074360,ARR - Storm Debris Collection,Phone,Closed,03/04/2023 09:50:42 AM,02/08/2023 05:20:01 PM,03/04/2023 09:50:42 AM,03/04/2023 09:50:42 AM,"6817 WALEBRIDGE LN, AUSTIN, TX 78739",6817.0,...,78739.0,TRAVIS,3.065975e+06,1.004126e+07,30.189998,-97.897932,"(30.18999782, -97.89793204)",8.0,671B,MA15
1346491,22-00331717,ARR - Garbage,Phone,Closed,09/14/2022 07:58:26 AM,09/08/2022 10:18:01 AM,09/14/2022 07:58:26 AM,09/14/2022 07:58:26 AM,"912 DALTON LN, AUSTIN, TX 78742",912.0,...,78742.0,TRAVIS,3.137499e+06,1.005805e+07,30.231641,-97.670344,"(30.2316411, -97.67034387)",3.0,616Z,MM19
721558,17-00062904,ARR Dead Animal Collection,Phone,Closed,03/14/2017 12:03:54 PM,03/13/2017 06:17:25 PM,03/14/2017 12:03:55 PM,03/14/2017 12:03:54 PM,"BOLM RD & PERRY RD, AUSTIN, TX 78721",NaN,...,78721.0,TRAVIS,3.132142e+06,1.006697e+07,30.256517,-97.686632,"(30.25651747, -97.68663211)",3.0,616C,MM21


In [30]:
sample_data = sample_data["SR Description"]
sample_data.to_csv(r"C:\Users\7J6594897\Desktop\100samples.csv")

In [31]:
#Procceded to make modifications in Excel and called modified data 100samples_labeled.csv

In [32]:
label_file_path = r"C:\Users\7J6594897\Desktop\100samples_labeled.csv"
labeled_data = pd.read_csv(label_file_path)
labeled_data.head()

,SR Description,Category
0,Debris in Street,Waste Collection and Recycling
1,ARR Missed Yard Trimmings/Compost,Tree Issues ans trimming
2,Wildlife Exposure,Animal Services
3,Tree Issue ROW,Tree Issues ans trimming
4,Found Animal Report - Keep,Animal Services


In [33]:
# Basic Text Preprocessor
class BasicTextPreprocessor(TransformerMixin):
    def transform(self, X, **transform_params):
        return [self.basic_preprocess(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def basic_preprocess(self, text):
        # Convert to lowercase
        text = text.lower()
        # Remove special characters
        text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A)
        # Simple tokenization (splitting text into words)
        tokens = text.split()
        return ' '.join(tokens)

model_data = labeled_data

# Split the data into training and testing sets
train_data, test_data = train_test_split(model_data, test_size=0.2, random_state=42)

# Create a pipeline with text preprocessing, TF-IDF vectorization, and SVM classifier
pipeline = make_pipeline(
    BasicTextPreprocessor(),
    TfidfVectorizer(),
    SVC(kernel='linear')  # Using a linear kernel for the SVM
)

# Train the model
pipeline.fit(train_data['SR Description'], train_data['Category'])

# Evaluate the model
predicted_categories = pipeline.predict(test_data['SR Description'])
report = classification_report(test_data['Category'], predicted_categories)

print(report)

                                        precision    recall  f1-score   support

                       Animal Services       1.00      1.00      1.00         9
                       Code Compliance       1.00      1.00      1.00         9
                         Public Safety       1.00      1.00      1.00         5
                  Shared Micromobility       1.00      1.00      1.00         1
Traffic and Road Infrastructure issues       1.00      1.00      1.00         4
             Tree Issues ans trimming        1.00      1.00      1.00         2
        Waste Collection and Recycling       1.00      1.00      1.00         3
                                 other       1.00      1.00      1.00         1

                              accuracy                           1.00        34
                             macro avg       1.00      1.00      1.00        34
                          weighted avg       1.00      1.00      1.00        34



In [34]:
complete_data = pd.read_csv(file_path)
# Check if the column name for SR Descriptions in the new dataset is the same or adjust accordingly
# Predict the categories for the new dataset
new_data_predictions = pipeline.predict(complete_data['SR Description'])  # Replace 'SR Description' if column name is different

# Add the predictions as a new column to the new dataset
complete_data['Predicted Category'] = new_data_predictions

# Save the new dataset with predictions to a new file
complete_data.to_csv(r'C:\Users\7J6594897\Desktop\test100samples_labeled.csv', index=False)


C:\Users\7J6594897\AppData\Local\Temp\ipykernel_17468\2205534670.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  complete_data = pd.read_csv(file_path)


In [35]:
complete_data[['SR Description','Predicted Category']]

,SR Description,Predicted Category
0,Animal - Proper Care,Animal Services
1,Tree Issue ROW,Tree Issues ans trimming
2,Pothole Repair,Traffic and Road Infrastructure issues
3,ARR Dead Animal Collection,Animal Services
4,Austin Code - Request Code Officer,Code Compliance
...,...,...
1716414,WPD - Lady Bird Lake Debris Issues,Waste Collection and Recycling
1716415,ACD - Request Code Officer,Code Compliance
1716416,ARR - Compost,other
1716417,ARR - Brush,Waste Collection and Recycling


In [36]:
complete_data['SR Description'].nunique()

314

In [37]:
complete_data['Predicted Category'].nunique()

9